In [1]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas scikeras

  Using cached scikeras-0.13.0-py3-none-any.whl.metadata (3.1 kB)
INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


In [2]:
!conda create -n tf-gpu --yes

!conda activate tf-gpu --yes

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/tf-gpu



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate tf-gpu
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications import EfficientNetB0

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
conda init bash

no change     /home/ec2-user/anaconda3/condabin/conda
no change     /home/ec2-user/anaconda3/bin/conda
no change     /home/ec2-user/anaconda3/bin/conda-env
no change     /home/ec2-user/anaconda3/bin/activate
no change     /home/ec2-user/anaconda3/bin/deactivate
no change     /home/ec2-user/anaconda3/etc/profile.d/conda.sh
no change     /home/ec2-user/anaconda3/etc/fish/conf.d/conda.fish
no change     /home/ec2-user/anaconda3/shell/condabin/Conda.psm1
no change     /home/ec2-user/anaconda3/shell/condabin/conda-hook.ps1
no change     /home/ec2-user/anaconda3/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /home/ec2-user/anaconda3/etc/profile.d/conda.csh
no change     /home/ec2-user/.bashrc
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!conda activate tf-gpu
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define the data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should not be augmented, only rescaled
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    'images/training',  # Path to your training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'images/testing',  # Path to your validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
def create_baseline_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(128, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_baseline_model, input_shape=(224, 224, 3), num_classes=3)

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.05, mode='max', verbose=1)


In [ ]:
param_grid = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__l2_reg': [0.0001, 0.001, 0.01],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad'],
    'fit__epochs': [2],
    'fit__batch_size': [32, 64]
}


In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=1)
grid_search_result = grid_search.fit(train_generator, validation_data=test_generator)

In [19]:
!conda update -n base -c conda-forge conda

Solving environment: / ^C
failed with repodata from current_repodata.json, will retry with next repodata source.


In [12]:
conda info -e

# conda environments:
#
base                     /home/ec2-user/anaconda3
JupyterSystemEnv         /home/ec2-user/anaconda3/envs/JupyterSystemEnv
R                        /home/ec2-user/anaconda3/envs/R
python3                  /home/ec2-user/anaconda3/envs/python3
pytorch_p310          *  /home/ec2-user/anaconda3/envs/pytorch_p310
tensorflow2_p310         /home/ec2-user/anaconda3/envs/tensorflow2_p310
tf-gpu                   /home/ec2-user/anaconda3/envs/tf-gpu


Note: you may need to restart the kernel to use updated packages.


In [14]:
!conda init bash
!conda activate tf-gpu --yes

no change     /home/ec2-user/anaconda3/condabin/conda
no change     /home/ec2-user/anaconda3/bin/conda
no change     /home/ec2-user/anaconda3/bin/conda-env
no change     /home/ec2-user/anaconda3/bin/activate
no change     /home/ec2-user/anaconda3/bin/deactivate
no change     /home/ec2-user/anaconda3/etc/profile.d/conda.sh
no change     /home/ec2-user/anaconda3/etc/fish/conf.d/conda.fish
no change     /home/ec2-user/anaconda3/shell/condabin/Conda.psm1
no change     /home/ec2-user/anaconda3/shell/condabin/conda-hook.ps1
no change     /home/ec2-user/anaconda3/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /home/ec2-user/anaconda3/etc/profile.d/conda.csh
no change     /home/ec2-user/.bashrc
No action taken.

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' fo

In [16]:
import subprocess

def check_gpu_usage():
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

check_gpu_usage()


Tue Jun 18 01:18:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
pip install gpustat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.4 MB/s eta 0:00:00
  Created wheel for gpustat: filename=gpustat-1.1.1-py3-none-any.whl size=26532 sha256=b3013ec1dffcc5ed1b253685f701ca2400dd752955e76fea1fbf8873e3187b6c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ec/d7/80/a71ba3540900e1f276bcae685efd8e590c810d2108b95f1e47
Successfully built gpustat
Note: you may need to restart the kernel to use updated packages.


In [21]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available: 1


In [23]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())


CUDA Available: True
GPU Count: 1


In [24]:
import tensorflow as tf

# Check GPU availability
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# Create a simple TensorFlow session
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

# Print the result
print("Matrix multiplication result:", c)


Num GPUs Available: 1


2024-06-18 01:25:51.650325: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 01:25:51.650619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 01:25:51.650769: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Matrix multiplication result: tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
